In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

# शिक्षक नेटवर्क और छात्र नेटवर्क
from nets.resnet import resnet32x4, resnet8x4
#ज्ञान आसवन का हानि कार्य केडी
from loss.dkd import dkd_loss

#टेन्सरबोर्ड
Train_Info = "DKD : Res32x4 To Res8x4"
writer = SummaryWriter(comment=Train_Info)

In [ ]:
# यादृच्छिक संख्या बीज सेट करें ताकि इसे पुन: प्रस्तुत किया जा सके
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(42)

In [ ]:
# GPU, अपने डिवाइस पर '0' नंबर को GPU नंबर से बदलें
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## हाइपरपैरामीटर को परिभाषित करें

In [ ]:
T = 4               # temperature : 知识蒸馏中的温度
ALPHA = 1.0         # alpha : TCKD 部分的loss weight
BETA = 8.0          # beta : NCKD 部分的loss weight
LOSS_CE = 1.0       # loss_ce : 交叉熵的loss weight
N = 100             # num_classes : 类别数
EPOCH = 20          # epoch : 训练轮数
BATCH_SIZE = 128    # batch_size : 批处理大小 
LR = 0.05           # learning_rate : 初试学习率

# ऑप्टिमाइज़र में अन्य हाइपरपैरामीटर, जैसे गति, भार-क्षय, मील के पत्थर, गामा, आदि आमतौर पर शायद ही कभी बदले जाते हैं।
# जब EPOCH बदलता है, तो मील के पत्थर भी उसी के अनुसार बदलने चाहिए।

## शिक्षक मॉडल लोड करें और छात्र नेटवर्क को परिभाषित करें

In [ ]:
res32x4 = resnet32x4(num_classes=N)
ckpt = torch.load("checkpoints/teacher/ckpt_epoch_240.pth", map_location='cpu')
res32x4.load_state_dict(ckpt["model"])
res32x4 = nn.DataParallel(res32x4).cuda()
res32x4.eval()

res8x4 = resnet8x4(num_classes=N)
res8x4 = torch.nn.DataParallel(res8x4).cuda()

teacher_net = res32x4
student_net = res8x4

## डेटासेट लोड करें
जब आप पहली बार इसका उपयोग करेंगे, तो यह पहले डाउनलोड होगा यदि डाउनलोड धीमा है, तो आप डेटासेट को मैन्युअल रूप से डाउनलोड कर सकते हैं और इसे डेटा फ़ोल्डर में खींच सकते हैं

In [ ]:
# डेटासेट तैयार करें और उसे प्रीप्रोसेस करें
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 先四周填充0，在把图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),     # 图像一半的概率翻转，一半的概率不翻转
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)), #R,G,B每层的归一化用到的均值和方差
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

DATA_PATH = "data"
# प्रशिक्षण डेटासेट
# num_workers आम तौर पर सीपीयू के प्रदर्शन पर निर्भर करता है
trainset = torchvision.datasets.CIFAR100(root=DATA_PATH, train=True, download=True, transform=transform_train) 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=1)   
# डेटासेट का परीक्षण करें
testset = torchvision.datasets.CIFAR100(root=DATA_PATH, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1)


## अनुकूलक

आम तौर पर, विभिन्न पेपरों में, cifar-100 डेटासेट पर `युग` को `240` पर सेट किया जाता है

संबंधित `मील के पत्थर` का मान `[150, 180, 210]` है

यहां, केवल प्रदर्शन के लिए, सभी `युग` को `40` पर सेट किया गया है, और `मील के पत्थर` का मान `[15, 25, 35]` है

In [ ]:
optimizer = torch.optim.SGD(student_net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 25, 35], gamma=0.1)

## ट्रेन फ़ंक्शन और परीक्षण फ़ंक्शन

In [ ]:
# प्रशिक्षण सेट और परीक्षण सेट पर क्रमशः सर्वश्रेष्ठ एसीसी को परिभाषित करें, इसे वैश्विक चर के रूप में संशोधित करने के लिए ग्लोबल का उपयोग करें, और फिर प्रशिक्षण के दौरान इसे अपडेट करें
best_train_acc = 0
best_test_acc = 0

In [ ]:
from tqdm import tqdm

def train(epoch):
    global best_train_acc

# छात्र मॉडल को प्रशिक्षण मोड पर सेट करें
    student_net.train()

    print('\nEpoch: %d' % epoch)

    train_loss = 0
    correct = 0
    total = 0

# प्रगति पट्टी प्रदर्शित करने के लिए ट्रेनलोडर को tqdm से लपेटें
    with tqdm(trainloader, desc=f"Training Epoch {epoch}", total=len(trainloader)) as pbar:
        for batch_idx, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            optimizer.zero_grad()

            logits_student, _ = student_net(inputs)
            with torch.no_grad():
                logits_teacher, _ = teacher_net(inputs)

#कठिन हानि
            ce_loss = nn.CrossEntropyLoss()(logits_student, targets)
#नरम नुकसान
            kd_loss = loss(logits_student, logits_teacher, temperature=T)
            total_loss = ALPHA * ce_loss + BETA * kd_loss

            total_loss.backward()
            optimizer.step()

            train_loss += total_loss.item()
            _, predicted = logits_student.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

#अपडेटटेन्सरबोर्ड
            writer.add_scalar('Train/Accuracy', 100. * correct / total, batch_idx + (epoch - 1) * 782)
            writer.add_scalar('Train/Loss', total_loss.item(), batch_idx + (epoch - 1) * 782)

# प्रगति पट्टी के प्रत्यय को अद्यतन करने के लिए set_postfix का उपयोग करें
            pbar.set_postfix(loss=train_loss / (batch_idx + 1), acc=f"{100. * correct / total:.1f}%")

# यदि वर्तमान प्रशिक्षण सेट की सटीकता best_test_acc से अधिक है, तो best_test_acc को अपडेट करें
    acc = 100 * correct / total
    if acc > best_train_acc:
        best_train_acc = acc


In [ ]:
def test(net, epoch):
    global best_test_acc
    net.eval()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
# प्रगति पट्टी प्रदर्शित करने के लिए टेस्टलोडर को लपेटने के लिए tqdm का उपयोग करें
        with tqdm(testloader, desc=f"Testing Epoch {epoch}", total=len(testloader)) as pbar:
            for batch_idx, (inputs, targets) in enumerate(pbar):

                inputs, targets = inputs.cuda(), targets.cuda()
                logits_student, _ = net(inputs)

                loss = nn.CrossEntropyLoss()(logits_student, targets)

                test_loss += loss.item()
                _, predicted = logits_student.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

# tqdm प्रगति पट्टी के प्रत्यय में वर्तमान हानि और सटीकता प्रदर्शित करें
                pbar.set_postfix(loss=test_loss / (batch_idx + 1), acc=f"{100. * correct / total:.1f}%")

#अपडेटटेन्सरबोर्ड
                writer.add_scalar('Test/Accuracy', 100. * correct / total, batch_idx + (epoch - 1) * 157)
                writer.add_scalar('Test/Loss', loss.item(), batch_idx + (epoch - 1) * 157)

# वर्तमान परीक्षण सेट पर सटीकता की गणना करें
        acc = 100. * correct / total

# यदि वर्तमान परीक्षण सेट पर सटीकता best_test_acc से अधिक है, तो best_test_acc अपडेट करें
# और छात्र मॉडल को सेव करें
        if acc > best_test_acc:
            print('Saving..')
            torch.save(student_net, 'checkpoints/student/kd_res8x4.pth')
            best_test_acc = acc

## रेलगाड़ी

In [ ]:
for epoch in range(1, EPOCH + 1) :
    train(epoch)
    test(student_net, epoch)

# सीखने की दर अद्यतन करें
    scheduler.step()
